# Useful functions for managing point cloud and skeleton files

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pathlib import Path
import open3d as o3d
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import scipy.io
from queue import Queue
import math
import random

In [3]:
def PCD_to_OFF(input_path):
    '''Convert .pcd point cloud files to ASCII .off-like format.'''
    pcd_path = Path(input_path).expanduser().resolve()
    pcd_name = str(pcd_path.stem)
    output_path = pcd_path.with_suffix(".off") # by default, output saved to input directory
    
    pcd = o3d.io.read_point_cloud(str(pcd_path)) # read the pcd file
    pts = np.asarray(pcd.points)
    with open(str(output_path), 'w') as h:
        h.write("OFF\n")
        h.write(f"{len(pcd.points)} 0 0\n")
        for point in pts:
            h.write(f"{point[0]} {point[1]} {point[2]}\n")

def PCD_to_PLY(input_path):
    '''Convert .pcd point cloud files to ASCII .ply format.'''
    pcd_path = Path(input_path).expanduser().resolve()
    pcd_name = str(pcd_path.stem)
    output_path = pcd_path.with_suffix(".ply") # by default, output saved to input directory

    pcd = o3d.io.read_point_cloud(str(pcd_path)) # read the pcd file
    # note: by default, o3d.io.write_point_cloud() can create ply files, but the header is incompatible
    # with the L1-skeleton software, since o3d defines `property double x`, whereas L1 expects `property float x`, etc
    pts = np.asarray(pcd.points)
    norms = np.asarray(pcd.normals)
    with open(str(output_path), 'w') as h:
        h.write("ply\n")
        h.write("format ascii 1.0\n")
        h.write(f"element vertex {len(pcd.points)}\n")
        h.write("property float x\n")
        h.write("property float y\n")
        h.write("property float z\n")
        h.write("property float nx\n")
        h.write("property float ny\n")
        h.write("property float nz\n")
        h.write("end_header\n")
        for point, norm in zip(pts, norms):
            h.write(f"{point[0]} {point[1]} {point[2]} {norm[0]} {norm[1]} {norm[2]}\n")
            
def PCD_to_NPY(input_path):
    '''Convert .pcd point cloud files to binary .npy format.'''
    pcd_path = Path(input_path).expanduser().resolve()
    pcd_name = str(pcd_path.stem)
    output_path = pcd_path.with_suffix(".npy") # by default, output saved to input directory
    
    pcd = o3d.io.read_point_cloud(str(pcd_path)) # read the pcd file
    pts = np.asarray(pcd.points)
    np.save(str(output_path), pts)
    
def XYZ_to_PCD(input_path):
    '''Convert .xyz point cloud files to binary .pcd format.'''
    xyz_path = Path(input_path).expanduser().resolve()
    xyz_name = str(xyz_path.stem)
    output_path = xyz_path.with_suffix(".pcd") # by default, output saved to input directory
    
    xyz = o3d.io.read_point_cloud(str(xyz_path)) # read the xyz file
    o3d.io.write_point_cloud(str(output_path), xyz)
    
def read_fork(input_path):
    '''Read fork .txt files (of format: x,y,z,nx,ny,nz), and convert to ASCII .ply format.'''
    fork_path = Path(input_path).expanduser().resolve()
    fork_name = str(fork_path.stem)
    output_path = fork_path.with_suffix(".ply") # by default, output saved to input directory
    
    pts = []
    norms = []
    with open(str(fork_path), 'r') as h:
        for line in h:
            data = line.split(",")
            x = data[0].strip()
            y = data[1].strip()
            z = data[2].strip() # skip null 4th dimension
            nx = data[4].strip()
            ny = data[5].strip()
            nz = data[6].strip("\n").strip() # has trailing \n
            pts.append([float(x), float(y), float(z)])
            norms.append([float(nx), float(ny), float(nz)])
    
    with open(str(output_path), 'w') as h:
        h.write("ply\n")
        h.write("format ascii 1.0\n")
        h.write(f"element vertex {len(pts)}\n")
        h.write("property float x\n")
        h.write("property float y\n")
        h.write("property float z\n")
        h.write("property float nx\n")
        h.write("property float ny\n")
        h.write("property float nz\n")
        h.write("end_header\n")
        for point, norm in zip(pts, norms):
            h.write(f"{point[0]} {point[1]} {point[2]} {norm[0]} {norm[1]} {norm[2]}\n")
            
def PLY_to_PCD(input_path):
    '''Convert .ply files to binary .pcd format.'''
    ply_path = Path(input_path).expanduser().resolve()
    ply_name = str(ply_path.stem)
    output_path = ply_path.with_suffix(".pcd") # by default, output saved to input directory
    
    ply = o3d.io.read_point_cloud(str(ply_path)) # read the ply file
    o3d.io.write_point_cloud(str(output_path), ply)
    
def make_torus(n, p):
    '''Generate n points of a torus, with probability p of missing data.'''
    R = 5 # major radius
    r = 1 # minor radius
    
    if p == 0:
        suffix = '0'
    else:
        suffix = str(p).split('.')[1]
        if len(suffix) == 1:
            suffix = suffix + '0'
    
    dest = f"/Users/kianfaizi/Desktop/P3D/Torus/torus{n}_noise{suffix}.xyz"
    
#     fig = plt.figure(figsize=(12,12))
#     ax = fig.add_subplot(111, projection='3d')
    
    with open(dest, 'w') as h:
        for i in range(n):
            if random.random() < (1-p): # noise
                u = random.uniform(0,360)
                v = random.uniform(0,360)
                x = (R + r*math.cos(v))*math.cos(u)
                y = (R + r*math.cos(v))*math.sin(u)
                z = r*math.sin(v)   
                h.write(f"{x} {y} {z}\n")
                #ax.scatter(x, y, z)

    XYZ_to_PCD(dest)
    
def parse_mat(input_path):
    '''Read .mat skeleton files from Cao et al.'''
    mat_path = Path(input_path).expanduser().resolve()
    mat_name = str(mat_path.stem)
    output_path = Path(mat_path.parent, mat_name+"_laplacian.txt")
    
    mat = scipy.io.loadmat(str(mat_path))
    mat = (mat['P'])[-1][-1] # discard annotations
    
    G = nx.Graph()
    
    # add skeleton point coordinates
    for point in enumerate(mat[-3]):
        n = point[0]
        coords = point[1]
        if not np.isnan(coords[0]):
            G.add_node(n, pos=(coords[0], coords[1], coords[2]))

    # add edges based on adjacency matrix
    adj = pd.DataFrame(mat[-1])
    for i in range(adj.shape[0]): # rows
        for j in range(i+1, adj.shape[1]): # upper triangular cols
            if adj.iloc[i,j] == 1:
                G.add_edge(adj.index.values[i], adj.columns.values[j])

    # check that all nodes are connected (no gaps in skeleton)
    print(f"Connected components: {len(list(nx.connected_components(G)))}")
    print(f"Is tree: {nx.is_tree(G)}")
    
    make_file(G, output_path)
    
def parse_skel(input_path):
    '''Read .skel skeleton files from Huang et al.'''
    skel_path = Path(input_path).expanduser().resolve()
    skel_name = str(skel_path.stem)
    output_path = Path(skel_path.parent, skel_name+"_L1").with_suffix(".txt")
    
    with open(str(skel_path), 'r') as h:
        num_on = int(h.readline().split()[1])
        # skip ON section (input points/normals)
        for i in range(num_on+1):
            h.readline()
        
        # skip SN section (skeleton points before processing?)
        num_sn = int(h.readline().split()[1])
        for i in range(num_sn+1):
            h.readline()
        
        # CN section contains skeleton branches
        # seems like they are written in hierarchical order
        G = nx.Graph()
        
        num_cn = int(h.readline().split()[1])
        branches = []
        idx = 0 # node ids
        pt_ids = {} # node id mappings for graph
        
        for cnn in range(num_cn):
            num_cnn = int(h.readline().split()[1])
            pts = [] # ordering/hierarchy
            for i in range(num_cnn):
                pt = h.readline()
                if pt not in pt_ids.keys(): 
                    pt_ids[pt] = idx
                    idx += 1
                pts.append(pt)
            branches.append(pts)

        # add nodes
        for pt,idx in pt_ids.items():
            coords = (pt.strip('\n')).split('\t')
            G.add_node(idx, pos=(float(coords[0]), float(coords[1]), float(coords[2])))
        
        # add edges
        for branch in branches:
            for i in range(len(branch)-1):
                G.add_edge(pt_ids[branch[i]], pt_ids[branch[i+1]])
            
    # check that all nodes are connected (no gaps in skeleton)
    print(f"Connected components: {len(list(nx.connected_components(G)))}")
    print(f"Is tree: {nx.is_tree(G)}")
    
    make_file(G, output_path)
    
def parse_txt(input_path):
    '''Read .txt files I produced from MarcSchotman's implementation of SkelTree.'''
    txt_path = Path(input_path).expanduser().resolve()
    txt_name = str(txt_path.stem)
    output_path = Path(txt_path.parent, txt_name+"_skeltree").with_suffix(".txt")
    
    with open(str(txt_path), 'r') as h:
        G = nx.Graph()
        connections = {}
        for line in h:
            # split into: name, coords, neighbors
            info = line.strip("\n").split("\t")
            coords = info[1].strip("[]").split()
            coords = list(map(float, coords))
            G.add_node(info[0], pos=(coords[0], coords[1], coords[2]))
            neighbors = info[2].lstrip("dict_keys").strip("()[]").split(", ")
            neighbors = list(map(lambda x: x[1:-1], neighbors)) # remove extra quotes
            connections[info[0]] = neighbors
            
    # add edges
    for k,v in connections.items():
        for i in v:
            G.add_edge(k,i)
            
    # check that all nodes are connected (no gaps in skeleton)
    print(f"Connected components: {len(list(nx.connected_components(G)))}")
    print(f"Is tree: {nx.is_tree(G)}")
            
    make_file(G, output_path)
    
def make_file(G, output_path):
    '''Output skeleton into a P3D-readable file.'''
    # view input skeleton
    nx.draw_spectral(G, with_labels=True)

    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111, projection='3d')
    for node in G.nodes():
        curr = G.nodes[node]
        ax.scatter(curr['pos'][0], curr['pos'][1], curr['pos'][2])

    # find terminal nodes
    terminal_node_ids = [node for node, degree in G.degree() if degree == 1]
    
    # which node is the root? pick the one with the smallest y-value
    root_y = G.nodes[terminal_node_ids[0]]['pos'][1]
    root_id = terminal_node_ids[0]
    
    for i in terminal_node_ids:
        y = G.nodes[i]['pos'][1]
        if y < root_y:
            root_y = y
            root_id = i
    
    attr_mapping = {k:v['pos'] for k,v in G.nodes.data()} # save mapping of nodes:positions
    G = nx.dfs_tree(G, root_id) # convert G into an oriented directed graph, starting at root
    nx.set_node_attributes(G, attr_mapping, 'pos') # the conversion loses attr data, so add back

    # relabel nodes depth-first
    preorder = list(nx.dfs_preorder_nodes(G, root_id))
    node_mapping = {v:k for k,v in enumerate(preorder)} # old id:new id
    G = nx.relabel_nodes(G, node_mapping, copy=True)
    root_id = 0

    # list nodes in ascending order, per level
    level_orders_dict = nx.single_source_shortest_path_length(G, root_id) # id:level
    level_orders = [(v,k) for k,v in level_orders_dict.items()] # (level, id)        
    level_orders = sorted(level_orders) # ids ascending within each level
        
    with open(str(output_path), "w") as h:
        current_level = -1
        for depth, node in level_orders:
            if current_level != depth:
                current_level = depth
                h.write(f"## Level: {depth}\n")
                counter = 0
            coords = G.nodes[node]['pos']
            h.write(f"{coords[0]} {coords[1]} {coords[2]};".rstrip("\n"))
            children = sorted(list(G.successors(node))) # ids of children
            n = len(children)
            if n > 0:
                for i in range(n-1):
                    h.write(f" [{level_orders_dict[children[i]]},{counter}]".rstrip("\n"))
                    counter += 1
                h.write(f" [{level_orders_dict[children[n-1]]},{counter}] \n")
                counter += 1
            else:
                h.write("\n")